In [4]:
import fitsio
import numpy as np
import healpy as hp
import pandas as pd

# For the moment I'm using Pandas (based on numpy), which is a very useful tool for
#handling tables and data.
# I'll properly put them into classes later

In [5]:
#12GB file, be carefull

file_name = 'spAll-v5_10_0.fits'

In [6]:
#The resolution of the map

NSide = 2**5

In [7]:
# Columns we are interested in

fits = fitsio.FITS(file_name)
cols = ['RA','DEC','THING_ID','MJD','PLATE','FIBERID','BOSS_TARGET1','EBOSS_TARGET0','EBOSS_TARGET1']

In [11]:
%%time

#http://stackoverflow.com/questions/30283836/creating-pandas-dataframe-from-numpy-array-leads-to-strange-errors
d = {col: fits[1][col].read().byteswap().newbyteorder() for col in cols}

CPU times: user 19 s, sys: 57.7 s, total: 1min 16s
Wall time: 2min 56s


In [14]:
df = pd.DataFrame(d)
print 'number of rows =', len(df)

number of rows = 3008000


In [16]:
'''map celestial coordinates to angular'''
phi_rad   = lambda ra : ra*np.pi/180.
theta_rad = lambda dec: (90.0 - dec)*np.pi/180.

df['PIX'] = hp.ang2pix(NSide, theta_rad(df['DEC']), phi_rad(df['RA']))

In [17]:
df.head()

,BOSS_TARGET1,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,PLATE,RA,THING_ID,PIX
0,0,26.188042,0,2,1,57346,10000,30.012094,339167825,3403
1,0,26.582945,0,1024,2,57346,10000,31.282286,342410358,3403
2,0,26.608501,0,2,3,57346,10000,31.279594,342410449,3403
3,0,26.682522,0,2048,4,57346,10000,31.237546,342675982,3403
4,0,26.866054,0,2,5,57346,10000,31.394521,344112497,3275


In [19]:
# Number of unique pixels

uniq_pix = df['PIX'].unique()
len(uniq_pix)

3369

In [20]:
def selec_pix(n):
    '''Given a pixel number, return all its info'''
    return df[df['PIX'] == uniq_pix[n]]

In [24]:
# Just checking that all rows are contained within all pixels

rows = 0
for n in xrange(len(uniq_pix)):
    rows += len(selec_pix(n))
    
assert rows == len(df)

In [59]:
# indexing and sorting by index

data = df.set_index(['PIX','THING_ID']).sort_index()

### Big table sorted by pixel

In [71]:
data[:15]

BOSS_TARGET1        DEC  EBOSS_TARGET0  EBOSS_TARGET1  \
PIX THING_ID                                                             
391 -1                      0  68.346702              0              0   
     552588186            134  68.199567              0              0   
     552894248        1048576  68.278564              0              0   
     552894319  2199023796224  68.381197              0              0   
     552894473            134  68.277346              0              0   
     552921285              0  68.440430              0              0   
     552921544              0  68.386842              0              0   
     552921545  2199023796224  68.418125              0              0   
     552921563              0  68.402368              0              0   
     552921566              1  68.357937              0              0   
     552921572              1  68.453486              0              0   
     552940613            134  68.488600              0              0   
392 -1                      0  68.562908              0              0   
    -1                      0  68.893565              0              0   
    -1                      0  68.495927              0              0   

                FIBERID    MJD  PLATE          RA  
PIX THING_ID                                       
391 -1              626  56667   7115  175.265990  
     552588186      583  56667   7115  174.875167  
     552894248      588  56667   7115  175.119697  
     552894319      622  56667   7115  175.258222  
     552894473      631  56667   7115  175.292988  
     552921285      630  56667   7115  175.805436  
     552921544      632  56667   7115  175.593915  
     552921545      624  56667   7115  175.582630  
     552921563      634  56667   7115  175.677322  
     552921566      628  56667   7115  175.492622  
     552921572      674  56667   7115  175.635500  
     552940613      673  56667   7115  175.994150  
392 -1              746  56748   6966  183.601920  
    -1              782  56748   6966  184.126610  
    -1              788  56748   6966  184.119700

### Select  a certain pixel info

In [102]:
#only the first rows of pixel 393
data.query('PIX == 393')[:10]

BOSS_TARGET1        DEC  EBOSS_TARGET0  EBOSS_TARGET1  \
PIX THING_ID                                                             
393 -1                      0  68.301196              0              0   
    -1                      0  68.127796              0              0   
    -1                      0  68.081373              0              0   
    -1                      0  68.507991              0              0   
     552543528  2199023779840  68.063838              0              0   
     552543837            134  68.022368              0              0   
     552543846  2199023255552  68.006130              0              0   
     552560214  2199023255552  68.186432              0              0   
     552560414              1  68.105650              0              0   
     552560416            134  68.128577              0              0   

                FIBERID    MJD  PLATE          RA  
PIX THING_ID                                       
393 -1              548  56748   6964  191.470290  
    -1              556  56748   6964  191.628550  
    -1              558  56748   6964  191.613650  
    -1              584  56748   6964  192.094960  
     552543528      560  56748   6964  191.662181  
     552543837      597  56748   6964  191.961890  
     552543846      596  56748   6964  192.019011  
     552560214      542  56748   6964  191.483419  
     552560414      550  56748   6964  191.502624  
     552560416      545  56748   6964  191.513720

In [106]:
print '# of THING_IDs within this pixel =', len(data.loc[393].index)
print '# of unique THING_IDs within this pixel =', len(data.loc[393].index.unique())

# of THING_IDs within this pixel = 28
# of unique THING_IDs within this pixel = 25


### Select a pixel 393 with THING_ID different of -1, and how some info

In [134]:
data.query('PIX == 393 & THING_ID != -1')[['DEC','RA', 'BOSS_TARGET1']].head()

DEC          RA   BOSS_TARGET1
PIX THING_ID                                       
393 552543528  68.063838  191.662181  2199023779840
    552543837  68.022368  191.961890            134
    552543846  68.006130  192.019011  2199023255552
    552560214  68.186432  191.483419  2199023255552
    552560414  68.105650  191.502624              1

### Search for quasars

In [172]:
bit_boss = [10,11,12,13,14,15,16,17,18,19,40,41,42,43,44]
bit_eboss = [10,11,12,13,14,15,16,17,18]

def is_quasar(dataset, mask_bit):
    is_qso  = lambda b: ((data[dataset] & 2**b) > 0)
    all_qso = map(is_qso, mask_bit)
    return reduce(lambda x, y: x |  y , all_qso)

In [176]:
print 'total qso in BOSS_TARGET1 = ', is_quasar('BOSS_TARGET1', bit_boss).sum()
print 'total qso in EBOSS_TARGET0 = ', is_quasar('EBOSS_TARGET0', bit_eboss).sum()
print 'total qso in EBOSS_TARGET1 = ', is_quasar('EBOSS_TARGET1', bit_eboss).sum() 

total qso in BOSS_TARGET1 =  466098
total qso in EBOSS_TARGET0 =  54108
total qso in EBOSS_TARGET1 =  201742


In [186]:
data['BOSS1_is_qso'] = is_quasar('BOSS_TARGET1', bit_boss)
data['EBOSS0_is_qso'] = is_quasar('EBOSS_TARGET0', bit_eboss)
data['EBOSS1_is_qso'] = is_quasar('EBOSS_TARGET1', bit_eboss)

### Table showing which are qso s

In [188]:
data.query('PIX == 393')[['BOSS1_is_qso','BOSS_TARGET1','MJD','PLATE','FIBERID']]

BOSS1_is_qso   BOSS_TARGET1    MJD  PLATE  FIBERID
PIX THING_ID                                                     
393 -1                False              0  56748   6964      548
    -1                False              0  56748   6964      556
    -1                False              0  56748   6964      558
    -1                False              0  56748   6964      584
     552543528         True  2199023779840  56748   6964      560
     552543837        False            134  56748   6964      597
     552543846         True  2199023255552  56748   6964      596
     552560214         True  2199023255552  56748   6964      542
     552560414        False              1  56748   6964      550
     552560416        False            134  56748   6964      545
     552780233        False            134  56748   6964      593
     552780633         True  1099511627776  56748   6964      586
     552807554         True  2199023796224  56748   6964      557
     552807704        False        1048576  56748   6964      544
     552807712         True  2199023255552  56748   6964      554
     552807875        False            140  56748   6964      549
     552807877        False            134  56748   6964      553
     552807884        False              0  56748   6964      551
     552807935        False            134  56748   6964      591
     552832069        False            134  56748   6964      541
     552832175         True  3298535424000  56748   6964      546
     553037421        False            140  56748   6964      585
     553055588        False            134  56748   6964      555
     553055710        False            134  56748   6964      583
     553055777         True  2199023796224  56748   6964      559
     553055821         True  2199023255552  56748   6964      582
     553055887        False            134  56748   6964      587
     553056104        False              1  56748   6964      581